In [ ]:
!pip install audiomentations
!pip install librosa
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import os

from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#output_size = (input_size - kernel_size + 2*padding) / stride + 1

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)


In [ ]:
!cd drive/MyDrive/'472 Group project'/Project/Datasets && pwd

In [ ]:
path = '/content/drive/MyDrive/472 Group project/Project/Datasets'

!cd path && ls

os.chdir(path)
os.listdir()

In [ ]:
# #this allowed me to download dataset directly to google drive

# !cd /content/drive/MyDrive/CS472/FinalTesting/DataSets && wget https://zenodo.org/record/1290750/files/IRMAS-TrainingData.zip?download=1
# !cd /content/drive/MyDrive/CS472/FinalTesting/DataSets && mv 'IRMAS-TrainingData.zip?download=1' IRMAS-TrainingData.zip && unzip IRMAS-TrainingData.zip && rm -r IRMAS-TrainingData.zip


In [ ]:
! pwd

In [ ]:
!ls

In [ ]:

def get_wf_sr(file_name):
    effects = [["remix", "1"]]
    return torchaudio.sox_effects.apply_effects_file(file_name, effects=effects)


def convert_waveform_to_mfcc(waveform, sr):
    n_fft = 2048
    win_length = None
    hop_length = 512
    n_mels = 256
    n_mfcc = 128
    mfcc_transform = T.MFCC(
        sample_rate=sr,
        n_mfcc=n_mfcc,
        melkwargs={
          'n_fft': n_fft,
          'n_mels': n_mels,
          'hop_length': hop_length,
          'mel_scale': 'htk',
        }
    )
    return mfcc_transform(waveform)

In [ ]:
class Cycler:
    def __init__(self):
      self.c = []
      self.i = 0


    def fromList(self, c_list):
        self.c = c_list
        self.i = 0

    def next(self):
      if len(self.c) <= self.i:
        return None
      else:
        item = self.c[self.i]
        self.i += 1
        return item

    def skip(self, num):
      self.i += num

    def reset(self):
      self.i = 0

    def __getitem__(self, idx):
      return self.c[idx]

    def __len__(self):
      return len(self.c)


    def toList(self):
        return self.c



In [ ]:
from re import I
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import librosa
import math
import random
import audiomentations as A
import wave
import torch.nn.functional as F

import librosa
import librosa.display
import matplotlib.pyplot as plt


from scipy.io import wavfile as wav




class IRMASDataset(Dataset):
    def __init__(self, root_dir, included_labels = [], segment_percent= .1, n_fts=128, train_percentage= .9, noise_percentage= .1, use_random=True):
        self.root_dir = root_dir
        self.use_random = use_random #this won't work if its not random 155

        self.included_labels = included_labels


        # shift will acquire the number of splits for audio file
        self.shift = int(1/ segment_percent) # also num_features
        self.segment_percent = segment_percent
        self.num_features = n_fts

        # datastructures
        self.file_list = []
        self.instrument_dict = {}
        self.train_data_dict = {}
        self.val_data_dict = {}
        self.test_data_dict = {}
        self.val_seg_list = []
        self.test_seg_list = []
        self.train_seg_list = []

        self.first_file = None

        #Information for dataloader partitioning
        self.val_percentage = .3 * train_percentage #change this to increase validation percentage
        self.train_percentage = train_percentage - (self.val_percentage)# change this to change everything else
        self.test_percentage = 1 - train_percentage

        #Noise augmentation
        self.noise_aug = A.Compose([A.AddGaussianSNR(p=noise_percentage,
                                              min_snr_in_db=10,
                                              max_snr_in_db=20)])
        #initialization functions
        self.initialize_data_structs()
        self.separate_train_test()

        #data structures/values that need to be initialized after above functions
        self.train_cut_off = int(len(self.file_list) * self.train_percentage)
        self.val_cut_off = int(len(self.file_list) * self.val_percentage) + self.train_cut_off
    def initialize_data_structs(self):
        instrument_dict = {}
        i = 0
        for root, dirs, files in os.walk(self.root_dir):
            for file in files:
                if file.endswith('.wav'):
                    file_name = os.path.join(root, file)
                    parent_dir = Path(file_name).parent.name
                    if parent_dir not in self.instrument_dict and parent_dir in self.included_labels:
                        self.instrument_dict[parent_dir] = i
                        i += 1
                        self.train_data_dict[parent_dir] = []
                        self.train_data_dict[parent_dir].append(file_name); self.file_list.append(file_name)
                    elif parent_dir in self.instrument_dict:
                        self.train_data_dict[parent_dir].append(file_name); self.file_list.append(file_name)
        if self.use_random:
            for instrument in self.train_data_dict:
               random.shuffle(self.train_data_dict[instrument])
            random.shuffle(self.file_list)

    def separate_train_test(self):
        for instrument in self.train_data_dict:
          train_cut_off = int(len(self.train_data_dict[instrument]) * self.train_percentage)
          val_cut_off = int(len(self.train_data_dict[instrument]) * self.val_percentage) + train_cut_off
          q1 = Cycler(); q1.fromList(self.train_data_dict[instrument][:train_cut_off])
          q2 = Cycler(); q2.fromList(self.train_data_dict[instrument][train_cut_off + 1:val_cut_off])
          q3 = Cycler(); q3.fromList(self.train_data_dict[instrument][val_cut_off +1:])
          self.train_data_dict[instrument] = q1
          self.val_data_dict[instrument] = q2
          self.test_data_dict[instrument] = q3

    def get_segment_list(self, target_dict):
        segment_list = []
        for instrument in target_dict:
            file_name = target_dict[instrument].next()
            if file_name == None:
                continue
            if self.first_file == None:
                self.first_file = file_name
            label = self.instrument_dict[instrument]
            label = torch.tensor(label)
            audio, sr = librosa.load(file_name, sr=None)
            mfcc = librosa.feature.mfcc(y = audio, sr=sr, n_mfcc=self.num_features)
            mfcc = librosa.power_to_db(mfcc, ref=np.max) #am trying this out
            mfcc = (mfcc - np.mean(mfcc)) / np.std(mfcc)
            mfcc = torch.tensor(mfcc, dtype=torch.float32)
            num_frames = mfcc.shape[1]
            segment_length = int(num_frames * self.segment_percent)  # number of frames per chunk

            # split the tensor into segments of length segment_length along the second dimension (i.e., dim=1)
            mfcc_frames_tensor = torch.chunk(mfcc, chunks= mfcc.shape[1] // segment_length, dim=1)
            for frame in mfcc_frames_tensor:
                if frame.shape[1] == segment_length and frame.shape[0] == self.num_features:
                    segment = frame.clone().detach().float().unsqueeze(0)
                    segment_list.append( (segment, label) )

        if len(segment_list) == 0:
            self.reset_cycles(target_dict)
            return self.get_segment_list(target_dict)
        return segment_list

    def get_first_file(self):
      return self.first_file

    def reset_cycles(self, target_dict):
      for instrument in target_dict:
        target_dict[instrument].reset()


    def full_reset(self):
      self.train_seg_list.clear()
      self.val_seg_list.clear()
      self.test_seg_list.clear()
      self.reset_cycles(self.train_data_dict)
      self.reset_cycles(self.val_data_dict)
      self.reset_cycles(self.test_data_dict)


    def __getitem__(self, idx):
        idx = idx // self.shift
        if idx <= self.train_cut_off:
          if len(self.train_seg_list) == 0:
            self.train_seg_list = self.get_segment_list(self.train_data_dict)
          mfcc_tensor, label_tensor = self.train_seg_list.pop()
          return mfcc_tensor, label_tensor
        elif idx > self.train_cut_off and idx <= self.val_cut_off:
          if len(self.val_seg_list) == 0:
            self.val_seg_list = self.get_segment_list(self.val_data_dict)
          mfcc_tensor, label_tensor = self.val_seg_list.pop()
          return mfcc_tensor, label_tensor
        else:
          if len(self.test_seg_list) == 0:
            self.test_seg_list = self.get_segment_list(self.test_data_dict)
          mfcc_tensor, label_tensor = self.test_seg_list.pop()
          return mfcc_tensor, label_tensor


    def __len__(self):
      return len(self.file_list) * self.shift

    def increment_cycles(self, splits):
      for inst in self.train_data_dict:
        increment = len(self.train_data_dict[inst]) // splits
        self.train_data_dict[inst].skip(increment)
      for inst in self.val_data_dict:
        increment = len(self.val_data_dict[inst]) // splits
        self.val_data_dict[inst].skip(increment)
      for inst in self.test_data_dict:
        increment = len(self.test_data_dict[inst]) // splits
        self.test_data_dict[inst].skip(increment)

    def get_label(self, filename):
        return self.instrument_dict[Path(filename).parent.name]

    def get_number_of_labels(self):
        return len(self.instrument_dict)

labels = ["sax", "tru", "vio", "voi", "cel", "cla", "flu", "gac", "gel", "org", "pia"]

IRMAS_ds = IRMASDataset("IRMAS-TrainingData", included_labels= labels)



In [ ]:
# # used to combine
def collate_fn(batch):
  inputs, targets = zip(*batch) #separtes data and target tensors into separate lists. Target tensors are the labels
  inputs_padded = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True) #padd the batches with zero if they are not filled
  targets_padded = torch.stack(targets) #padds the targets with 0's until it is filled
  return inputs_padded, targets_padded

In [ ]:
from torch.utils.data import Sampler
class StartIndexSampler(Sampler):
    def __init__(self, start_index, end_index):
        self.start_index = start_index
        self.end_index = end_index

    def __iter__(self):
        return iter(range(self.start_index, self.end_index))

    def __len__(self):
        return self.end_index - self.start_index

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler, SequentialSampler, Subset

import copy

torch.manual_seed(42)


def split_samples(dataset, num_splits=10, batch_size = 64, num_workers= 2):
  num_files = len(dataset)
  dataset.full_reset()

  dataset_copies = []
  train_dsLoader_list = []
  val_dsLoader_list = []
  test_dsLoader_list = []
  train_percentage = dataset.train_percentage
  val_percentage = dataset.val_percentage
  test_percentage = dataset.test_percentage

  train_start = 0
  val_start = dataset.train_cut_off * dataset.shift + dataset.shift
  test_start = dataset.val_cut_off * dataset.shift + dataset.shift


  train_increment = int(train_percentage* num_files // num_splits)
  val_increment = int(val_percentage * num_files // num_splits)
  test_increment = int(test_percentage * num_files // num_splits)

  train_prev = train_start
  val_prev = val_start
  test_prev = test_start

  train_next = train_prev + train_increment
  val_next = val_prev + val_increment
  test_next = test_prev + test_increment
  print(f"split {num_files} inputs by {num_splits} splits: train size= {train_increment}, validation size= {val_increment}, test size= {test_increment}")
  skip_len = 0
  current_dataset = copy.deepcopy(dataset)
  for i in range(num_splits):
      train_dsLoader_list.append( DataLoader(current_dataset,
                                            num_workers=num_workers,
                                            batch_size= batch_size,
                                            sampler= StartIndexSampler(train_prev, train_next),
                                            collate_fn= collate_fn
                                            ) )
      train_prev = train_next
      train_next += train_increment
      #print(val_prev, val_next)
      val_dsLoader_list.append(DataLoader(current_dataset,
                                          num_workers=num_workers,
                                          batch_size= batch_size,
                                          sampler= StartIndexSampler(val_prev, val_next),
                                          collate_fn= collate_fn
                                          ) )
      val_prev = val_next + 1
      val_next += val_increment
      test_dsLoader_list.append( DataLoader(current_dataset,
                                            num_workers=num_workers,
                                            batch_size= batch_size,
                                            sampler= StartIndexSampler(test_prev, test_next),
                                            collate_fn= collate_fn
                                            ) )
      test_prev = test_next + 1
      test_next += test_increment
      dataset_copies.append(current_dataset)
      current_dataset = copy.deepcopy(current_dataset)
      current_dataset.increment_cycles(num_splits)

  return train_dsLoader_list, val_dsLoader_list, test_dsLoader_list, dataset_copies



In [ ]:
from typing_extensions import dataclass_transform
batch_size = 1
train_dsLoader_list, val_dsLoader_list, test_dsLoader_list, dataset_copies = split_samples(IRMAS_ds, num_splits = 1, batch_size= batch_size) #dataset, num_workers =2, num_splits, batch_size = 2


#data, label = next(iter(test_dsLoader_list[0]))
# data1, label1 = next(iter(train_dsLoader_list[5]))
# print(data[0][0][30])
# print(data1[0][0][30])




Test to show the data sets are not equal, each time it a dataloader
is called it copies the current dataset state it is assigned to by
making copies it refers to the state of the copy.

Copy of output:

tensor([0.1227, 0.0205, 0.0053, 0.1575, 0.1030, 0.0833, 0.1713, 0.1905, 0.1395, 0.1874, 0.1687, 0.0704, 0.0523])

tensor([-0.4809, -0.5016, -0.5025, -0.4433, -0.3915, -0.3718, -0.2472, -0.2602, -0.2802, -0.2363, -0.1246, -0.1997, -0.1865])

In [ ]:
sample_idx = 0 # index of the sample to retrieve
print(f"Number of instruments: {len(IRMAS_ds.instrument_dict)}\n")
#print(IRMAS_ds.data_dict.keys())
print(f"Shape of output: {IRMAS_ds[0][0].shape} \n")  #torch.Size([1, 50, 13])
print(f"Number of wave files: {len(IRMAS_ds) // 10}\n")

#if you set segment_percentage to .1 the resulting shape will be 26 by 26.


train_keys = "Train keys: "
for key in IRMAS_ds.train_data_dict:
  train_keys += (f" {key}:{len(IRMAS_ds.train_data_dict[key])}")
print(train_keys)

val_keys = "\nVal keys: "
for key in IRMAS_ds.train_data_dict:
  val_keys += (f" {key}:{len(IRMAS_ds.val_data_dict[key])}")
print(val_keys)

test_keys = "\nTest keys: "
for key in IRMAS_ds.test_data_dict:
  test_keys += f" {key}:{len(IRMAS_ds.test_data_dict[key])} "
print(test_keys)




In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import random

import torchaudio
import torchaudio.transforms as T

file_name = IRMAS_ds.file_list[random.randint(0, len(IRMAS_ds.file_list) - 1)]
# load audio file
audio, sr = librosa.load(file_name)

# plot waveform
plt.figure(figsize=(14, 5))
librosa.display.waveshow(audio, sr=sr)
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.title('Waveform plot')
plt.show()


def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)










# for instrument in IRMAS_ds.train_data_dict:
#   # sample = cycle.next()
#   print(instrument)
#   file_name = IRMAS_ds.train_data_dict[instrument].next()
#   wf, sr = get_wf_sr(file_name)
#   mfcc = convert_waveform_to_mfcc(wf, sr)
#   plot_spectrogram(mfcc[0])



In [ ]:
# #verify there is data in the the test and train loaders, commented out as it takes a long time to run

# data, label = next(iter(train_dsLoader_list[0]))
# print(f"train targets: {label}, {data.shape}, {len(data)}")

# data, label = next(iter(val_dsLoader_list[0]))
# print(f"val targets: {label}, {data.shape}, {len(data)}")

# data, label = next(iter(test_dsLoader_list[0]))
# print(f"test targets: {label}, {data.shape}, {len(data)}")



output_length = (input_length + 2 * padding - dilation * (kernel_size -1) - 1) / stride + 1

output_shape = (batch_size, out_channels, output_length)


In [ ]:
class LinearNet(nn.Module):
    def __init__(self,
                 ninputs= 780,   # number of inputs (30x30)
                 nhidden=100,   # number of nodes in hidden layer
                 nout=11,       # output
                 nLayers = 2,
                 channels = 1,
                 dropout_prob=0.1
                ):
        super().__init__()
        layers = [nn.Linear(ninputs, nhidden), nn.BatchNorm1d(nhidden), nn.ReLU(), nn.Dropout(p=dropout_prob)]

        for i in range(nLayers):
            layers += [nn.Linear(nhidden, nhidden), nn.BatchNorm1d(nhidden), nn.ReLU(), nn.Dropout(p=dropout_prob)]

        layers += [nn.Linear(nhidden, nout, bias=False)]

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        # reshape the input tensor to have four dimensions
        b, c, h, w = x.shape
        x = x.view(b, -1) # Flatten image

        # pass the reshaped tensor through the neural network
        x = self.net(x)

        return x


In [ ]:
class ResBlock(nn.Module):
    def __init__(self, inChannels, outChannels, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(inChannels,
                               outChannels,
                               kernel_size=3,
                               stride=stride,
                               padding=1,
                               bias=False)
        self.bn = nn.BatchNorm2d(outChannels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.downsample = downsample


    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=11, dropout_prob=0.5):
        super(ResNet, self).__init__()
        self.inChannels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, outChannels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inChannels != outChannels:
            downsample = nn.Sequential(
                nn.Conv2d(self.inChannels, outChannels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outChannels),
            )

        layers = []
        layers.append(block(self.inChannels, outChannels, stride, downsample))
        self.inChannels = outChannels
        for i in range(1, blocks):
            layers.append(block(self.inChannels, outChannels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)

        return x


In [ ]:
class tmp_ds(Dataset):
    def __init__(self, val_images, val_labels):
        self.images = val_images
        self.labels = val_labels

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

    def __len__(self):
        return len(self.images)

In [ ]:
import time
import matplotlib.pyplot as plt
from tqdm import tqdm # This is optional but useful
import gc

CUDA_LAUNCH_BLOCKING=1

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'



class Trainer():
    def __init__(self, model, dataset, num_splits= 20, batch_size=512, num_epochs= 10, num_workers= 1):
        #hyper Parameters
        self.num_epochs = num_epochs
        self.lr = .001
        self.batch_size = (batch_size // dataset.shift) * dataset.shift  #ensures that each batch size does not cut between instruments
        self.num_splits = num_splits

        item = dataset[0]
        plot_spectrogram(item[0][0])

        self.train_list, self.val_list, self.test_list, self.dataset_copies = split_samples(dataset,
                                                                                            num_splits = self.num_splits,
                                                                                            batch_size= self.batch_size,
                                                                                            num_workers= num_workers)

        self.training_data = []
        self.training_labels = []
        self.testing_data = []
        self.testing_labels = []


        self.dataset = dataset
        self.total_loss = []
        self.total_val_loss = []

        self.reuse_val_dataset = None
        self.reuse_val_loader = None

        self.num_labels = dataset.get_number_of_labels()
        self.num_channels = 0
        self.num_frames = 0
        self.num_features =  0
        self.get_data_info()
        self.model = model
        #self.model = model(ResBlock, [3,4,6,3], num_classes= num_labels).to(device)
        self.criterion = nn.CrossEntropyLoss()

        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr, weight_decay= .001, momentum= .09)
        #weight decay is done to prevent overfitting, it does this by adding a penalty to the loss function to make it have smaller values
        #momentum helps smooth out the update process and make learning happen quicker

    def get_data_info(self):
        extract_mfcc_tensor, label = self.dataset[0]
        self.num_channels, self.num_frames, self.num_features = extract_mfcc_tensor.shape

    def train_over_partition(self, partition_num):
        if partition_num > self.num_splits or partition_num < 0:
            print(f"num_partition must be >=0 and  < {self.num_splits}")
        train_loader = self.train_list[partition_num]
        valid_loader = self.val_list[partition_num]
        test_loader = self.test_list[partition_num]

        self.model.train()
        loss_history = []
        val_loss_history = []
        loss = torch.Tensor([0])

        start_time = time.time()
        reuse_images = []
        reuse_labels = []
        first_pass = True
        for epoch in tqdm(range(self.num_epochs), desc=f"Epoch", unit="epoch", disable=False):
            #train on train data
            for (images, labels) in tqdm(train_loader, desc="iteration", unit="%", disable=True):
                self.optimizer.zero_grad(set_to_none=True)
                images = images.to(device)
                labels = labels.to(device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                loss_history.append(loss.item())
                del images, labels, outputs
                torch.cuda.empty_cache()
                gc.collect()
            # Validation
            self.model.eval()
            with torch.no_grad():
                for (images, labels) in valid_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = self.model(images)
                    if first_pass:
                        reuse_images.extend(images)
                        reuse_labels.extend(labels)
                    val_loss = self.criterion(outputs, labels)
                    val_loss_history.append(val_loss.item())
            first_pass = False
            self.model.train()

            # Print epoch and losses
            avg_train_loss = sum(loss_history) / len(loss_history)
            avg_val_loss = sum(val_loss_history) / len(val_loss_history)
            print(f"Epoch {epoch}: train_loss: {avg_train_loss:.5f}, val_loss: {avg_val_loss:.5f}") # glitch in which it doesn't print last print output

        #Train on validation data
        print("\nTraining over Validation set:")
        for epoch in tqdm(range(self.num_epochs), desc=f"Epoch", unit="epoch", disable=False):
            self.reuse_val_data(reuse_images, reuse_labels)
            for (images, labels) in tqdm(self.reuse_val_loader, desc="iteration", unit="%", disable=True):
                self.optimizer.zero_grad(set_to_none=True)
                images = images.to(device)
                labels = labels.to(device)
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                loss_history.append(loss.item())
                del images, labels, outputs
                torch.cuda.empty_cache()
                gc.collect()
            avg_train_loss = sum(loss_history) / len(loss_history)
            print(f"Epoch {epoch}: train_loss: {avg_train_loss:.5f}")
        end_time = time.time()
        print(f"Total training time: {(end_time - start_time) / 60:.2f} minutes")

        self.total_loss.extend(loss_history)
        self.total_val_loss.extend(val_loss_history)
        plt.plot(self.total_loss, label='Training set')
        plt.plot(self.total_val_loss, label='Validation set')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

        correct = 0
        total = 0
        with torch.no_grad():
          for data in self.test_list[partition_num]:
              inputs, labels = data
              inputs = inputs.to(device)
              labels = labels.to(device)
              outputs = self.model(inputs)
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
        print(f"Accuracy of the network on the test set: {round((100 * correct / total), 2)}%\n")

    def get_eval_data_over_range(self, start, end):
        #collecting training data
        print("\nCollecting output data for training")
        with torch.no_grad():
          for i in range(start, end):
              for data in self.train_list[i]:
                  inputs, labels = data
                  inputs = inputs.to(device)
                  labels = labels.to(device)
                  outputs = self.model(inputs)
                  self.collect_training_data(outputs, labels)
              for data in self.val_list[i]:
                  inputs, labels = data
                  inputs = inputs.to(device)
                  labels = labels.to(device)
                  outputs = self.model(inputs)
                  self.collect_training_data(outputs, labels)
              for data in self.test_list[i]:
                  inputs, labels = data
                  inputs = inputs.to(device)
                  labels = labels.to(device)
                  outputs = self.model(inputs)
                  self.collect_training_data(outputs, labels, if_test=True)


    def collect_training_data(self, outputs, labels, if_test=False):
        tmp_list = []
        for i, output in enumerate(F.softmax(outputs, dim=1)):
            tmp_list.append(output)
            if (i + 1) % self.dataset.shift == 0:
              if if_test == True:
                  self.testing_labels.append(labels[i].to("cpu"))
                  self.testing_data.append(torch.stack(tmp_list).unsqueeze(0).to("cpu"))
              else:
                  self.training_labels.append(labels[i].to("cpu"))
                  self.training_data.append(torch.stack(tmp_list).unsqueeze(0).to("cpu"))
              tmp_list.clear()


    def get_training_data(self):
        return self.training_data, self.training_labels, self.testing_data, self.testing_labels


    def reuse_val_data(self,images, labels):
        self.reuse_val_dataset = tmp_ds(images, labels)
        self.reuse_val_loader = DataLoader(self.reuse_val_dataset, batch_size= self.batch_size, shuffle=True)

        # temp = list(zip(images, labels))
        # random.shuffle(temp)
        # r_images, r_labels = zip(*temp)


    def only_test(self):
        correct = 0
        total = 0
        with torch.no_grad():
          for test_partition in self.test_list:
            for data in test_partition:
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f"Accuracy of the network on the test set: {round((100 * correct / total), 2)}%\n")

    def train_over_range(self, start, end):
        for i in range(start, end + 1):
            print(f"\npartition: {i}/{end}\n")
            self.train_over_partition(i)
        self.get_eval_data_over_range(start, end + 1)



In [ ]:
# model = LinearNet().to(device)
#model = CNNModel(11).to(device)
model = ResNet(ResBlock, [3, 4, 6, 3]).to(device)


labels = ["sax", "tru", "vio", "voi", "cel", "cla", "flu", "gac", "gel", "org", "pia"]
IRMAS_ds = IRMASDataset("IRMAS-TrainingData", segment_percent= .1, included_labels= labels)


trainer = Trainer(model, IRMAS_ds, num_splits= 1, num_epochs= 30)
trainer.train_over_partition(0)
#trainer.train_over_range(0,0)


In [ ]:
data, labels, test_data, test_labels = trainer.get_training_data()
results_ds = tmp_ds(data, labels)
test_ds = tmp_ds(test_data, test_labels)

print("checking training data")
print(len(data))
print(len(labels))

print("checking testing data")
print(len(test_data))
print(len(test_labels))

# print(results_ds[0][0])






In [ ]:
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler


# Define indices for train, validation, and test splits
num_samples = len(results_ds)
num_test_samples = len(test_ds)

train_split = int(num_samples * 0.9)
val_split = num_samples - train_split

train_indices = list(range(train_split))
val_indices = list(range(train_split, train_split + val_split))
test_indices = list(range(0, num_test_samples))

# Define samplers for each split
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Define data loaders for each split
batch_size = 64
train_loader = DataLoader(results_ds, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(results_ds, batch_size=batch_size, sampler=val_sampler)
test_loader = DataLoader(test_ds, batch_size=batch_size, sampler=test_sampler)


#print(next(iter(train_loader))[0].shape)


In [ ]:

# model =  LinearNet(
#                  ninputs= 10 * 11,   # number of inputs (30x30)
#                  nhidden=100,   # number of nodes in hidden layer
#                  nout=11,       # output
#                  nLayers = 1,
#                  channels = 1,
#                  dropout_prob=0.1
#                 ).to(device)


model = ResNet(ResBlock, [3, 4, 6, 3]).to(device)

num_epochs = 100 #50 looks optimal

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= .001, weight_decay= .001, momentum= .09)

model.train()
loss_history = []
val_loss_history = []
loss = torch.Tensor([0])

reuse_images = []
reuse_labels = []
first_pass = True
start_time = time.time()
for epoch in tqdm(range(num_epochs), desc=f"Epoch", unit="epoch", disable=False):
    #train on train data
    for (images, labels) in tqdm(train_loader, desc="iteration", unit="%", disable=True):
        optimizer.zero_grad(set_to_none=True)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    # Validation
    model.eval()
    with torch.no_grad():
        for (images, labels) in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            val_loss_history.append(val_loss.item())
    model.train()
    avg_train_loss = sum(loss_history) / len(loss_history)
    avg_val_loss = sum(val_loss_history) / len(val_loss_history)
    print(f"Epoch {epoch}: train_loss: {avg_train_loss:.5f}, val_loss: {avg_val_loss:.5f}") # glitch in which it doesn't print last print output

#Train on validation data
print("\nTraining over Validation set:")
for epoch in tqdm(range(num_epochs//3), desc=f"Epoch", unit="epoch", disable=False):
    for (images, labels) in tqdm(val_loader, desc="iteration", unit="%", disable=True):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        loss_history.append(loss.item())
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
    avg_train_loss = sum(loss_history) / len(loss_history)
    print(f"Epoch {epoch}: train_loss: {avg_train_loss:.5f}")

#total_loss.extend(loss_history)
#total_val_loss.extend(val_loss_history)
plt.plot(loss_history, label='Training set')
plt.plot(val_loss_history, label='Validation set')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


end_time = time.time() # End time
print(f"Total training time: {(end_time - start_time) / 60:.2f} minutes")

correct = 0
total = 0
with torch.no_grad():
  for (images, labels) in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
print(f"Accuracy of the network on the test set: {round((100 * correct / total), 2)}%\n")

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for (images, labels) in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
print(f"Accuracy of the network on the test set: {round((100 * correct / total), 2)}%\n")

In [ ]:
# model = CNN(11).to(device)
# trainer = Trainer(model, IRMAS_ds, num_splits= 5, num_epochs= 10)
# trainer.train_over_range(0, 4) #error here, original had set to 5, but that would be 6 partitions

In [ ]:
#model = ResNet(ResBlock, [2, 2, 2, 2]).to(device) #68%
# labels = ["sax", "tru", "vio", "voi", "cel", "cla", "flu", "gac", "gel", "org", "pia"]
# import torch.nn as nn

# import torch.nn as nn

# Define the CNN model



# train_labels = []
# for label in labels:
#     train_labels.append(label)
#     if len(train_labels) == 2:

#         train_labels.clear()


In [ ]:
# labels = ["sax", "tru", "vio", "voi", "cel", "cla", "flu", "gac", "gel", "org"] # no piano
# IRMAS_ds = IRMASDataset("IRMAS-TrainingData", included_labels= labels)
# trainer = Trainer(model, IRMAS_ds, num_splits= 10, num_epochs= 10)
# trainer.only_test()

In [ ]:

        # temp = list(zip(images, labels))
        # random.shuffle(temp)
        # r_images, r_labels = zip(*temp)